In [1]:
import sys
import os
sys.path.append('/root/capsule/code/beh_ephys_analysis')
from harp.clock import decode_harp_clock, align_timestamps_to_anchor_points
from open_ephys.analysis import Session
import datetime
from aind_ephys_rig_qc.temporal_alignment import search_harp_line
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.colors import LinearSegmentedColormap
import pandas as pd
from pynwb import NWBFile, TimeSeries, NWBHDF5IO
from scipy.io import loadmat
from scipy.stats import zscore
import ast
from utils.plot_utils import combine_pdf_big

from open_ephys.analysis import Session
from pathlib import Path
import glob

import json
import seaborn as sns
from PyPDF2 import PdfMerger
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import re
from aind_dynamic_foraging_basic_analysis.plot.plot_foraging_session import plot_foraging_session
from aind_dynamic_foraging_data_utils.nwb_utils import load_nwb_from_filename
from hdmf_zarr.nwb import NWBZarrIO
from utils.beh_functions import session_dirs, parseSessionID, load_model_dv, makeSessionDF, get_session_tbl, get_unit_tbl, get_history_from_nwb
from utils.ephys_functions import*
import pandas as pd
import pickle
import scipy.stats as stats
from joblib import Parallel, delayed
from multiprocessing import Pool
from functools import partial
import time
import spikeinterface as si
import shutil
import seaborn as sns
import math
import seaborn as sns
from sklearn.decomposition import PCA
from scipy.stats import zscore
%matplotlib inline

In [8]:
# Make combined session-unit table
df = pd.read_csv('/root/capsule/code/data_management/session_assets.csv')
session_ids = df['session_id'].values
behs = df['behavior'].values
exclude = ['ecephys_717120_2024-03-06_12-23-53', 'ecephys_713854_2024-03-08_14-54-25', 'ecephys_713854_2024-03-08_16-20-33', 'behavior_754897_2025-03-15_11-32-18', 'behavior_752014_2025-03-28_11-04-59']
session_ids, behs = zip(*[
    (session, beh)
    for session, beh in zip(session_ids, behs)
    if isinstance(session, str) and session not in exclude
])
session_ids = list(session_ids)
behs = list(behs)

In [9]:
# loop through all sessions
# count number of units pass qc and opto qc
# check if exist in longer than 100 sessions 
# check if exist in pre/post stimulation
# check if exist in anti-dromic stimulation
all_sessions = []
all_units = []
all_qc_pass_units = [] # default qc pass
all_real_units = [] # not noise, not artifact
all_tagged_units = []
in_behavior = []
trial_count = []
opto_tagging_pre = []
opto_tagging_post = []
anti = []
probes = []
all_p_max = []
all_lat_max_p = []  # latency of max p response
isi_v = []
all_eu = []
all_corr = []
all_amp = []
all_peak = []
all_wf = []
all_wf_aligned = []
all_wf_2d = []
y_loc = []
in_df = []
rec_side = []
top = []
bottom = []
snr = []

# p_resp_thresh = 0.5
# lat_resp_thresh = 0.02

target = 'soma'
for session, beh in zip(session_ids, behs):
    session_dir = session_dirs(session)
    if session_dir['curated_dir_curated'] is not None:
        if 'behavior_758017_2025-02-06_11-26-14' in session:
            print('behavior_758017_2025-02-06_11-26-14')
        print(f'Processing {session}')
        data_type = 'curated'
        # if session_dir['curated_dir_curated'] is not None:
        #     data_type = 'curated'
        # elif session_dir['curated_dir_raw'] is not None:
        #     data_type = 'raw'
        # else:
        #     continue

        unit_tbl = get_unit_tbl(session, data_type)
        all_units.extend(unit_tbl['unit_id'].tolist())
        all_qc_pass_units.extend(unit_tbl['default_qc'].tolist())  # default qc pass
        # all_real_units.extend(unit_tbl['real_unit'].tolist())
        all_tagged_units.extend(unit_tbl['tagged_loc'].tolist())  # tagged location (e.g. 'soma', 'axon', 'unspecified')
        all_sessions.extend([session]*len(unit_tbl))
        if 'p_max' not in unit_tbl.columns:
            all_p_max.extend(unit_tbl['p_max_x'].tolist())
            all_lat_max_p.extend(unit_tbl['lat_max_p_x'].tolist())
            all_eu.extend(unit_tbl['euc_max_p_x'].tolist())
            all_corr.extend(unit_tbl['corr_max_p_x'].tolist())
            peaks = unit_tbl['peak_x'].values
            amp = unit_tbl['amp_x'].values
        else: 
            all_p_max.extend(unit_tbl['p_max'].tolist())  
            all_lat_max_p.extend(unit_tbl['lat_max_p'].tolist()) 
            all_eu.extend(unit_tbl['euc_max_p'].tolist())
            all_corr.extend(unit_tbl['corr_max_p'].tolist()) 
            peaks = unit_tbl['peak'].values
            amp = unit_tbl['amp'].values
     
        isi_v.extend(unit_tbl['isi_violations_ratio'].tolist())  # ISI violations
        snr.extend(unit_tbl['snr'].tolist())  # signal-to-noise ratio
        y_loc.extend(unit_tbl['y_loc'].tolist())  # y location of the unit

        if 'peak_wf_opt' in unit_tbl.columns:
            wf_opt = [wf_opt_unit if isinstance(wf_opt_unit, np.ndarray) else wf_unit for wf_opt_unit, wf_unit in zip(unit_tbl['peak_wf_opt'], unit_tbl['peak_wf'])]  # peak waveform
            wf_opt_aligned = [wf_opt_unit if isinstance(wf_opt_unit, np.ndarray) else wf_unit for wf_opt_unit, wf_unit in zip(unit_tbl['peak_wf_opt_aligned'], unit_tbl['peak_wf_aligned'])]  # peak waveform aligned
            wf_opt_2d = [wf_opt_unit if isinstance(wf_opt_unit, np.ndarray) else wf_unit for wf_opt_unit, wf_unit in zip(unit_tbl['mat_wf_opt'], unit_tbl['wf_2d'])]  # peak waveform 2D
        else:
            wf_opt = [unit_tbl['peak_wf'].values]
            wf_opt_aligned = [unit_tbl['peak_wf_aligned'].values]
            wf_opt_2d = [unit_tbl['wf_2d'].values]

        amp_opt = [
                        np.max(wf_opt_curr) - np.min(wf_opt_curr) if isinstance(wf_opt_curr, np.ndarray) else curr_amp_unit
                        for wf_opt_curr, curr_amp_unit in zip(wf_opt, amp)
                    ]   # amplitude of optimized waveforms
        peak_opt = [
                        curr_peak_opt_unit if not np.isnan(curr_peak_opt_unit) else curr_peak_unit
                        for curr_peak_opt_unit, curr_peak_unit in zip(list(unit_tbl['amplitude_opt'].values), list(peaks))
                    ]
        
        all_amp.extend(amp_opt)  # amplitude of max p response
        all_peak.extend(peak_opt)  # peak opto response
        all_wf.extend(wf_opt)  # waveform of max p response
        all_wf_aligned.extend(wf_opt_aligned)  # waveform of max p response
        all_wf_2d.extend(wf_opt_2d)  # waveform of max p response
        top.extend(unit_tbl['LC_range_top'].tolist())
        bottom.extend(unit_tbl['LC_range_bottom'].tolist())

        # session and opto information
        session_df = get_session_tbl(session)
        with open(os.path.join(session_dir[f'opto_dir_{data_type}'], f'{session}_opto_info_{target}.json')) as f:
            opto_info = json.load(f)
        opto_df = pd.read_csv(os.path.join(session_dir[f'opto_dir_{data_type}'], f'{session}_opto_session_{target}.csv'))
        if len(opto_df[opto_df['pre_post'] == 'pre'])>0:
            pre_end = np.max(opto_df[opto_df['pre_post'] == 'pre']['time'].values)
        else:
            pre_end = np.nan
        
        if len(opto_df[opto_df['pre_post'] == 'post'])>0:
            post_start = np.min(opto_df[opto_df['pre_post'] == 'post']['time'].values)
            post_end = np.max(opto_df[opto_df['pre_post'] == 'post']['time'].values)
        else:
            post_start = np.nan
            post_end = np.nan
            
        for unit_id in unit_tbl['unit_id'].values:
            curr_pre_opto = True
            curr_post_opto = True 
            curr_anti_opto = True  
            if session_df is not None:
                curr_in_beh = True
            else:
                curr_in_beh = False
                curr_trial_count = 0
            unit = unit_tbl[unit_tbl['unit_id'] == unit_id]
            unit_drift = load_drift(session, unit_id, data_type=data_type)
            if session_df is not None:
                go_cue_times = session_df['goCue_start_time']
                if unit_drift is not None:
                    if unit_drift['ephys_cut'][0] is not None:
                        if unit_drift['ephys_cut'][0] > pre_end - 2*60:
                            curr_pre_opto = False 
                        go_cue_times = go_cue_times[go_cue_times >= unit_drift['ephys_cut'][0]]         
                    if unit_drift['ephys_cut'][1] is not None:
                        if unit_drift['ephys_cut'][1] < post_start + 2*60:
                            curr_post_opto = False
                        if unit_drift['ephys_cut'][1] < post_end + 2*60:
                            curr_anti_opto = False
                        go_cue_times = go_cue_times[go_cue_times <= unit_drift['ephys_cut'][1]]
                if len(go_cue_times) < 100: 
                    curr_in_beh = False
                curr_trial_count = len(go_cue_times)
            opto_tagging_pre.append(curr_pre_opto)
            opto_tagging_post.append(curr_post_opto)
            anti.append(curr_anti_opto)
            trial_count.append(curr_trial_count)
            in_behavior.append(curr_in_beh)
        probes.extend([df[df['session_id']==session]['probe'].values[0]] * len(unit_tbl))
        rec_side.extend([df[df['session_id']==session]['side'].values[0]]*len(unit_tbl))  # recording side
        in_df.extend([beh] * len(unit_tbl))  # store session df for each unit
combined_tagged_units = pd.DataFrame({'session': all_sessions,
                                        'unit': all_units,
                                        'qc_pass': all_qc_pass_units,
                                        'opto_tagged': all_tagged_units,
                                        'opto_tagging_pre': opto_tagging_pre,
                                        'opto_tagging_post': opto_tagging_post,
                                        'anti': anti,
                                        'in_df': in_behavior,
                                        'trial_count': trial_count,
                                        'p_max': all_p_max,
                                        'lat_max_p': all_lat_max_p,
                                        'isi_violations': isi_v,
                                        'snr': snr,
                                        'eu': all_eu,
                                        'corr': all_corr,
                                        'amp': all_amp,
                                        'peak': all_peak,
                                        'wf': all_wf,
                                        'wf_aligned': all_wf_aligned,
                                        'wf_2d': all_wf_2d,
                                        'probe': probes,
                                        'y_loc': y_loc, 
                                        'rec_side': rec_side,
                                        'top': top,
                                        'bottom': bottom,
                                        })

# save dataframe in combined folder
with open(os.path.join('/root/capsule/scratch/combined/combine_unit_tbl', 'combined_unit_tbl.pkl'), 'wb') as f:
    pickle.dump(combined_tagged_units, f)

Processing ecephys_713854_2024-03-05_12-01-40
Processing ecephys_713854_2024-03-05_13-01-09
Processing ecephys_713854_2024-03-05_13-31-20
Processing ecephys_717120_2024-03-06_12-54-27
Processing ecephys_717120_2024-03-07_12-12-02
Processing ecephys_713854_2024-03-08_15-43-01
There is no nwb file in the curated directory.
Processing ecephys_684930_2023-09-27_10-04-04
Processing ecephys_684930_2023-09-28_11-45-27
Processing ecephys_684930_2023-09-28_12-44-15
Processing ecephys_687697_2023-09-15_11-30-06
Processing ecephys_687697_2023-09-15_12-36-06
Processing ecephys_691893_2023-10-05_12-46-57
Processing ecephys_691893_2023-10-06_13-48-18
Processing behavior_716325_2024-05-31_10-31-14
Processing behavior_717121_2024-06-15_10-00-58
There are multiple recordings in the raw nwb directory. Please specify the recording you would like to use.
Processing behavior_751004_2024-12-19_11-50-37
Processing behavior_751004_2024-12-20_13-26-11
Processing behavior_751004_2024-12-21_13-28-28
Processing b

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [10]:
# loop through all sessions
# count number of units pass qc and opto qc
# check if exist in longer than 100 sessions 
# check if exist in pre/post stimulation
# check if exist in anti-dromic stimulation
all_sessions = []
all_units = []
all_qc_pass_units = [] # default qc pass
all_real_units = [] # not noise, not artifact
all_tagged_units = []
in_behavior = []
trial_count = []
opto_tagging_pre = []
opto_tagging_post = []
anti = []
probes = []
all_p_max = []
all_lat_max_p = []  # latency of max p response
isi_v = []
all_eu = []
all_corr = []
all_amp = []
all_peak = []
all_wf = []
all_wf_aligned = []
all_wf_2d = []
y_loc = []
in_df = []
rec_side = []
top = []
bottom = []
snr = []

# p_resp_thresh = 0.5
# lat_resp_thresh = 0.02

target = 'soma'
for session, beh in zip(session_ids, behs):
    session_dir = session_dirs(session)
    if session_dir['curated_dir_curated'] is not None:
        if 'behavior_758017_2025-02-06_11-26-14' in session:
            print('behavior_758017_2025-02-06_11-26-14')
        print(f'Processing {session}')
        data_type = 'curated'
        # if session_dir['curated_dir_curated'] is not None:
        #     data_type = 'curated'
        # elif session_dir['curated_dir_raw'] is not None:
        #     data_type = 'raw'
        # else:
        #     continue

        unit_tbl = get_unit_tbl(session, data_type)
        all_units.extend(unit_tbl['unit_id'].tolist())
        all_qc_pass_units.extend(unit_tbl['default_qc'].tolist())  # default qc pass
        # all_real_units.extend(unit_tbl['real_unit'].tolist())
        all_tagged_units.extend(unit_tbl['tagged_loc'].tolist())  # tagged location (e.g. 'soma', 'axon', 'unspecified')
        all_sessions.extend([session]*len(unit_tbl))
        if 'p_max' not in unit_tbl.columns:
            all_p_max.extend(unit_tbl['p_max_x'].tolist())
            all_lat_max_p.extend(unit_tbl['lat_max_p_x'].tolist())
            all_eu.extend(unit_tbl['euc_max_p_x'].tolist())
            all_corr.extend(unit_tbl['corr_max_p_x'].tolist())
            peaks = unit_tbl['peak_x'].values
            amp = unit_tbl['amp_x'].values
        else: 
            all_p_max.extend(unit_tbl['p_max'].tolist())  
            all_lat_max_p.extend(unit_tbl['lat_max_p'].tolist()) 
            all_eu.extend(unit_tbl['euc_max_p'].tolist())
            all_corr.extend(unit_tbl['corr_max_p'].tolist()) 
            peaks = unit_tbl['peak'].values
            amp = unit_tbl['amp'].values
     
        isi_v.extend(unit_tbl['isi_violations_ratio'].tolist())  # ISI violations
        snr.extend(unit_tbl['snr'].tolist())  # signal-to-noise ratio
        y_loc.extend(unit_tbl['y_loc'].tolist())  # y location of the unit

        top.extend(unit_tbl['LC_range_top'].tolist())
        bottom.extend(unit_tbl['LC_range_bottom'].tolist())

        # session and opto information
        session_df = get_session_tbl(session)
        with open(os.path.join(session_dir[f'opto_dir_{data_type}'], f'{session}_opto_info_{target}.json')) as f:
            opto_info = json.load(f)
        opto_df = pd.read_csv(os.path.join(session_dir[f'opto_dir_{data_type}'], f'{session}_opto_session_{target}.csv'))
        if len(opto_df[opto_df['pre_post'] == 'pre'])>0:
            pre_end = np.max(opto_df[opto_df['pre_post'] == 'pre']['time'].values)
        else:
            pre_end = np.nan
        
        if len(opto_df[opto_df['pre_post'] == 'post'])>0:
            post_start = np.min(opto_df[opto_df['pre_post'] == 'post']['time'].values)
            post_end = np.max(opto_df[opto_df['pre_post'] == 'post']['time'].values)
        else:
            post_start = np.nan
            post_end = np.nan
            
        for unit_id in unit_tbl['unit_id'].values:
            curr_pre_opto = True
            curr_post_opto = True 
            curr_anti_opto = True  
            if session_df is not None:
                curr_in_beh = True
            else:
                curr_in_beh = False
                curr_trial_count = 0
            unit = unit_tbl[unit_tbl['unit_id'] == unit_id]
            unit_drift = load_drift(session, unit_id, data_type=data_type)
            if session_df is not None:
                go_cue_times = session_df['goCue_start_time']
                if unit_drift is not None:
                    if unit_drift['ephys_cut'][0] is not None:
                        if unit_drift['ephys_cut'][0] > pre_end - 2*60:
                            curr_pre_opto = False 
                        go_cue_times = go_cue_times[go_cue_times >= unit_drift['ephys_cut'][0]]         
                    if unit_drift['ephys_cut'][1] is not None:
                        if unit_drift['ephys_cut'][1] < post_start + 2*60:
                            curr_post_opto = False
                        if unit_drift['ephys_cut'][1] < post_end + 2*60:
                            curr_anti_opto = False
                        go_cue_times = go_cue_times[go_cue_times <= unit_drift['ephys_cut'][1]]
                if len(go_cue_times) < 100: 
                    curr_in_beh = False
                curr_trial_count = len(go_cue_times)
            opto_tagging_pre.append(curr_pre_opto)
            opto_tagging_post.append(curr_post_opto)
            anti.append(curr_anti_opto)
            trial_count.append(curr_trial_count)
            in_behavior.append(curr_in_beh)
        probes.extend([df[df['session_id']==session]['probe'].values[0]] * len(unit_tbl))
        rec_side.extend([df[df['session_id']==session]['side'].values[0]]*len(unit_tbl))  # recording side
        in_df.extend([beh] * len(unit_tbl))  # store session df for each unit
combined_tagged_units = pd.DataFrame({'session': all_sessions,
                                        'unit': all_units,
                                        'qc_pass': all_qc_pass_units,
                                        'opto_tagged': all_tagged_units,
                                        'opto_tagging_pre': opto_tagging_pre,
                                        'opto_tagging_post': opto_tagging_post,
                                        'anti': anti,
                                        'in_df': in_behavior,
                                        'trial_count': trial_count,
                                        'p_max': all_p_max,
                                        'lat_max_p': all_lat_max_p,
                                        'isi_violations': isi_v,
                                        'snr': snr,
                                        'eu': all_eu,
                                        'corr': all_corr,
                                        'probe': probes,
                                        'y_loc': y_loc, 
                                        'rec_side': rec_side,
                                        'top': top,
                                        'bottom': bottom,
                                        })

# save dataframe in combined folder
with open(os.path.join('/root/capsule/scratch/combined/combine_unit_tbl', 'combined_unit_tbl.pkl'), 'wb') as f:
    pickle.dump(combined_tagged_units, f)

Processing ecephys_713854_2024-03-05_12-01-40
Processing ecephys_713854_2024-03-05_13-01-09
Processing ecephys_713854_2024-03-05_13-31-20
Processing ecephys_717120_2024-03-06_12-54-27
Processing ecephys_717120_2024-03-07_12-12-02
Processing ecephys_713854_2024-03-08_15-43-01
There is no nwb file in the curated directory.
Processing ecephys_684930_2023-09-27_10-04-04
Processing ecephys_684930_2023-09-28_11-45-27
Processing ecephys_684930_2023-09-28_12-44-15
Processing ecephys_687697_2023-09-15_11-30-06
Processing ecephys_687697_2023-09-15_12-36-06
Processing ecephys_691893_2023-10-05_12-46-57
Processing ecephys_691893_2023-10-06_13-48-18
Processing behavior_716325_2024-05-31_10-31-14
Processing behavior_717121_2024-06-15_10-00-58
There are multiple recordings in the raw nwb directory. Please specify the recording you would like to use.
Processing behavior_751004_2024-12-19_11-50-37
Processing behavior_751004_2024-12-20_13-26-11
Processing behavior_751004_2024-12-21_13-28-28
Processing b